In [6]:
import psycopg2

# Load configuration
def load_config():
    import yaml
    with open('config.yaml', 'r') as file:
        return yaml.safe_load(file)

# Connect to the database
config = load_config()
db_config = config['DATABASE']
conn = psycopg2.connect(
    dbname=db_config['NAME'],
    user=db_config['USER'],
    password=db_config['PASSWORD'],
    host=db_config['HOST'],
    port=db_config['PORT']
)

# Create a cursor
cur = conn.cursor()

# Add the new column `dm_preference` if it doesn't exist
cur.execute("""
    DO $$
    BEGIN
        IF NOT EXISTS (
            SELECT 1
            FROM information_schema.columns
            WHERE table_name='birthdays' AND column_name='dm_preference'
        ) THEN
            ALTER TABLE birthdays ADD COLUMN dm_preference BOOLEAN DEFAULT FALSE;
        END IF;
    END $$;
""")

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

# Database Management for Birthday Bot
This notebook helps manage the database operations for the birthday bot.

In [2]:
import psycopg2
import yaml

def load_config():
    with open('config/config.yaml', 'r') as file:
        return yaml.safe_load(file)

def get_db_connection():
    config = load_config()
    db_config = config['DATABASE']
    return psycopg2.connect(
        dbname=db_config['NAME'],
        user=db_config['USER'],
        password=db_config['PASSWORD'],
        host=db_config['HOST'],
        port=db_config['PORT']
    )

# Create a fresh connection
conn = get_db_connection()
print("Database connection established")

Database connection established


In [3]:
# Check all birthdays in the database
cur = conn.cursor()

# Get table name from config
config = load_config()
table_name = config['DATABASE']['TABLE_NAME']

# Query all users and their birthdays
cur.execute(f"SELECT user_id, username, birthday, firstname, lastname, dm_preference FROM {table_name} ORDER BY birthday")
results = cur.fetchall()

print(f"Total users in {table_name}: {len(results)}")
print("\nAll users and birthdays:")
for row in results:
    user_id, username, birthday, firstname, lastname, dm_preference = row
    print(f"User ID: {user_id}, Username: {username}, Birthday: {birthday}, First: {firstname}, Last: {lastname}, DM: {dm_preference}")

# Specifically check for today (2025-09-03)
from datetime import datetime, date
today = date(2025, 9, 3)
print(f"\nChecking for birthdays on {today} (today)...")

# Check who has birthday today (month and day match)
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname, dm_preference 
    FROM {table_name} 
    WHERE EXTRACT(MONTH FROM birthday) = %s 
    AND EXTRACT(DAY FROM birthday) = %s
""", (today.month, today.day))

todays_birthdays = cur.fetchall()
print(f"Found {len(todays_birthdays)} birthdays today:")
for row in todays_birthdays:
    user_id, username, birthday, firstname, lastname, dm_preference = row
    print(f"🎂 User ID: {user_id}, Username: {username}, Birthday: {birthday}, First: {firstname}, Last: {lastname}, DM: {dm_preference}")

cur.close()

Total users in birthday_dev: 4

All users and birthdays:
User ID: 163035782259539968, Username: Husky, Birthday: 1998-09-07, First: None, Last: None, DM: False
User ID: 417445162487119874, Username: Kasane, Birthday: 2025-07-18, First: Tim, Last: None, DM: False
User ID: 1358045590222541052, Username: BirthdayBot-Dev, Birthday: None, First: None, Last: None, DM: False
User ID: 305068083347718145, Username: Piccar, Birthday: None, First: None, Last: None, DM: False

Checking for birthdays on 2025-09-03 (today)...
Found 0 birthdays today:


In [4]:
# Add birthday for September 3rd, 1998 to test today's birthday checking
from datetime import date

# Reconnect to database
conn = get_db_connection()
cur = conn.cursor()

# Let's add the birthday for September 3rd, 1998 to the Husky user (assuming that's your user)
user_id_to_update = 163035782259539968  # Husky's user ID
new_birthday = date(1998, 9, 3)  # September 3rd, 1998

print(f"Adding birthday {new_birthday} for user {user_id_to_update}...")

# Update the birthday
cur.execute(f"""
    UPDATE {table_name} 
    SET birthday = %s 
    WHERE user_id = %s
""", (new_birthday, user_id_to_update))

conn.commit()
print(f"✅ Birthday updated successfully!")

# Verify the change
cur.execute(f"SELECT user_id, username, birthday FROM {table_name} WHERE user_id = %s", (user_id_to_update,))
result = cur.fetchone()
if result:
    print(f"Verified: User {result[1]} (ID: {result[0]}) now has birthday: {result[2]}")

cur.close()
conn.close()

Adding birthday 1998-09-03 for user 163035782259539968...
✅ Birthday updated successfully!
Verified: User Husky (ID: 163035782259539968) now has birthday: 1998-09-03


In [5]:
# Now let's check if today's birthday detection works
conn = get_db_connection()
cur = conn.cursor()

today = date(2025, 9, 3)  # Today (September 3rd, 2025)
print(f"🎂 Checking for birthdays on {today} (today)...")

# Check who has birthday today (month and day match)
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname, dm_preference 
    FROM {table_name} 
    WHERE EXTRACT(MONTH FROM birthday) = %s 
    AND EXTRACT(DAY FROM birthday) = %s
""", (today.month, today.day))

todays_birthdays = cur.fetchall()
print(f"Found {len(todays_birthdays)} birthdays today:")
for row in todays_birthdays:
    user_id, username, birthday, firstname, lastname, dm_preference = row
    age = today.year - birthday.year
    print(f"🎉 User ID: {user_id}, Username: {username}, Birthday: {birthday}, Age: {age}, First: {firstname}, Last: {lastname}, DM: {dm_preference}")

cur.close()
conn.close()

if todays_birthdays:
    print(f"\n✅ SUCCESS: Found {len(todays_birthdays)} birthday(s) for today!")
    print("The database now correctly has a birthday for September 3rd.")
else:
    print(f"\n❌ No birthdays found for today ({today})")

🎂 Checking for birthdays on 2025-09-03 (today)...
Found 1 birthdays today:
🎉 User ID: 163035782259539968, Username: Husky, Birthday: 1998-09-03, Age: 27, First: None, Last: None, DM: False

✅ SUCCESS: Found 1 birthday(s) for today!
The database now correctly has a birthday for September 3rd.


In [6]:
# Add firstname and lastname for Husky to make birthday messages more complete
conn = get_db_connection()
cur = conn.cursor()

# Update Husky with proper name information
user_id_husky = 163035782259539968
firstname = "Klemens"
lastname = "Drobnicki"

print(f"Adding name information for Husky (ID: {user_id_husky})...")
print(f"Setting firstname: {firstname}, lastname: {lastname}")

cur.execute(f"""
    UPDATE {table_name} 
    SET firstname = %s, lastname = %s
    WHERE user_id = %s
""", (firstname, lastname, user_id_husky))

conn.commit()
print("✅ Name information updated successfully!")

# Verify the complete record
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname, dm_preference 
    FROM {table_name} 
    WHERE user_id = %s
""", (user_id_husky,))

result = cur.fetchone()
if result:
    user_id, username, birthday, firstname, lastname, dm_preference = result
    print(f"Verified complete record:")
    print(f"  User ID: {user_id}")
    print(f"  Username: {username}")
    print(f"  Birthday: {birthday}")
    print(f"  First name: {firstname}")
    print(f"  Last name: {lastname}")
    print(f"  DM preference: {dm_preference}")

cur.close()
conn.close()

Adding name information for Husky (ID: 163035782259539968)...
Setting firstname: Klemens, lastname: Drobnicki
✅ Name information updated successfully!
Verified complete record:
  User ID: 163035782259539968
  Username: Husky
  Birthday: 1998-09-03
  First name: Klemens
  Last name: Drobnicki
  DM preference: False


In [7]:
# Check current DM preferences and test the DM preference functionality
conn = get_db_connection()
cur = conn.cursor()

print("🔍 Current DM preferences for all users:")
cur.execute(f"SELECT user_id, username, dm_preference FROM {table_name} ORDER BY user_id")
dm_results = cur.fetchall()

for user_id, username, dm_preference in dm_results:
    print(f"  User: {username} (ID: {user_id}) - DM enabled: {dm_preference}")

print(f"\n📧 Users with DM preferences enabled:")
cur.execute(f"SELECT user_id, username FROM {table_name} WHERE dm_preference = TRUE")
dm_enabled_users = cur.fetchall()
print(f"Found {len(dm_enabled_users)} users with DM enabled:")
for user_id, username in dm_enabled_users:
    print(f"  - {username} (ID: {user_id})")

if len(dm_enabled_users) == 0:
    print("❌ No users have DM preferences enabled!")
    print("Let's enable DM preference for Husky to test...")
    
    # Enable DM preference for Husky
    husky_id = 163035782259539968
    cur.execute(f"UPDATE {table_name} SET dm_preference = TRUE WHERE user_id = %s", (husky_id,))
    conn.commit()
    print(f"✅ Enabled DM preference for Husky (ID: {husky_id})")
    
    # Verify the change
    cur.execute(f"SELECT username, dm_preference FROM {table_name} WHERE user_id = %s", (husky_id,))
    result = cur.fetchone()
    if result:
        username, dm_pref = result
        print(f"Verified: {username} now has DM preference: {dm_pref}")

cur.close()
conn.close()

🔍 Current DM preferences for all users:
  User: Husky (ID: 163035782259539968) - DM enabled: False
  User: Piccar (ID: 305068083347718145) - DM enabled: False
  User: Kasane (ID: 417445162487119874) - DM enabled: False
  User: BirthdayBot-Dev (ID: 1358045590222541052) - DM enabled: False

📧 Users with DM preferences enabled:
Found 0 users with DM enabled:
❌ No users have DM preferences enabled!
Let's enable DM preference for Husky to test...
✅ Enabled DM preference for Husky (ID: 163035782259539968)
Verified: Husky now has DM preference: True


In [8]:
# Test the get_users_with_dm_enabled method directly
conn = get_db_connection()
cur = conn.cursor()

print("🧪 Testing the exact query used by get_users_with_dm_enabled():")
cur.execute(f"SELECT user_id FROM {table_name} WHERE dm_preference = TRUE")
dm_users = cur.fetchall()
print(f"Found {len(dm_users)} users with DM enabled:")
for row in dm_users:
    user_id = row[0]
    print(f"  - User ID: {user_id}")

# Also get their names for context
if dm_users:
    user_ids = [str(row[0]) for row in dm_users]
    user_ids_str = ','.join(user_ids)
    cur.execute(f"SELECT user_id, username FROM {table_name} WHERE user_id IN ({user_ids_str})")
    users_info = cur.fetchall()
    print("\nWith usernames:")
    for user_id, username in users_info:
        print(f"  - {username} (ID: {user_id})")

cur.close()
conn.close()

if dm_users:
    print(f"\n✅ SUCCESS: The database query for DM-enabled users works correctly!")
    print(f"The bot should now find {len(dm_users)} user(s) with DM enabled when it runs.")
else:
    print(f"\n❌ ERROR: No DM-enabled users found, even though we just set one!")

🧪 Testing the exact query used by get_users_with_dm_enabled():
Found 1 users with DM enabled:
  - User ID: 163035782259539968

With usernames:
  - Husky (ID: 163035782259539968)

✅ SUCCESS: The database query for DM-enabled users works correctly!
The bot should now find 1 user(s) with DM enabled when it runs.


In [9]:
# Debug the DM sending and reaction handling issues

# First, let's check the exact format of get_users_with_dm_enabled() return values
conn = get_db_connection()
cur = conn.cursor()

print("🔍 Testing get_users_with_dm_enabled() return format:")
cur.execute(f"SELECT user_id FROM {table_name} WHERE dm_preference = TRUE")
notif_users = cur.fetchall()
print(f"Raw result: {notif_users}")
print(f"Type: {type(notif_users)}")

if notif_users:
    print(f"First item: {notif_users[0]}")
    print(f"Type of first item: {type(notif_users[0])}")
    
    # Test the unpacking logic used in birthday_service
    for notif_user_id, *_ in notif_users:
        print(f"Unpacked user_id: {notif_user_id} (type: {type(notif_user_id)})")

print("\n🧪 Testing the DM logic simulation:")
# Simulate what happens in _send_notifications
birthday_user_id = 163035782259539968  # Husky's ID
print(f"Birthday user ID: {birthday_user_id}")

for notif_user_id, *_ in notif_users:
    print(f"Checking if {notif_user_id} != {birthday_user_id}: {notif_user_id != birthday_user_id}")
    if notif_user_id != birthday_user_id:
        print(f"  ✅ Would send DM to user {notif_user_id}")
    else:
        print(f"  ❌ Skipping DM to birthday user {notif_user_id}")

# Let's also check if the notification message content and emoji are correct
print(f"\n📧 Checking notification message setup:")
print("Looking for a message containing 'Geburtstags-Benachrichtigungen' with ✅ reaction...")

cur.close()
conn.close()

🔍 Testing get_users_with_dm_enabled() return format:
Raw result: [(163035782259539968,)]
Type: <class 'list'>
First item: (163035782259539968,)
Type of first item: <class 'tuple'>
Unpacked user_id: 163035782259539968 (type: <class 'int'>)

🧪 Testing the DM logic simulation:
Birthday user ID: 163035782259539968
Checking if 163035782259539968 != 163035782259539968: False
  ❌ Skipping DM to birthday user 163035782259539968

📧 Checking notification message setup:
Looking for a message containing 'Geburtstags-Benachrichtigungen' with ✅ reaction...


In [ ]:
# Add birthday for Kasane today and enable DM preferences for testing
from datetime import date

conn = get_db_connection()
cur = conn.cursor()

# Kasane's user ID from earlier
kasane_id = 417445162487119874
today_birthday = date(1995, 9, 3)  # September 3rd, 1995 (different year for variety)

print(f"🎂 Adding birthday for Kasane (ID: {kasane_id}) on {today_birthday}...")

# Update Kasane's birthday to today and enable DM preference
cur.execute(f"""
    UPDATE {table_name} 
    SET birthday = %s, dm_preference = TRUE, firstname = %s, lastname = %s
    WHERE user_id = %s
""", (today_birthday, "Tim", "Kasane", kasane_id))

conn.commit()
print(f"✅ Set birthday and enabled DM preference for Kasane")

# Also enable DM preference for Piccar so we have someone to receive notifications
piccar_id = 305068083347718145
cur.execute(f"UPDATE {table_name} SET dm_preference = TRUE WHERE user_id = %s", (piccar_id,))
conn.commit()
print(f"✅ Enabled DM preference for Piccar (ID: {piccar_id})")

# Verify all changes
print(f"\n📊 Current status:")
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname, dm_preference 
    FROM {table_name} 
    ORDER BY birthday DESC NULLS LAST
""")
all_users = cur.fetchall()

for user_id, username, birthday, firstname, lastname, dm_pref in all_users:
    age = ""
    if birthday:
        age = f" (Age: {today.year - birthday.year})" if birthday.month == today.month and birthday.day == today.day else ""
    print(f"  {username} (ID: {user_id}) - Birthday: {birthday}{age}, DM: {dm_pref}, Name: {firstname} {lastname}")

# Check today's birthdays
print(f"\n🎉 Today's birthdays (September 3rd):")
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname, dm_preference 
    FROM {table_name} 
    WHERE EXTRACT(MONTH FROM birthday) = 9 
    AND EXTRACT(DAY FROM birthday) = 3
""")
todays_birthdays = cur.fetchall()

for user_id, username, birthday, firstname, lastname, dm_pref in todays_birthdays:
    age = today.year - birthday.year
    print(f"  🎂 {username} (ID: {user_id}) - {firstname} {lastname} turns {age}, DM enabled: {dm_pref}")

print(f"\n📧 Users with DM enabled:")
cur.execute(f"SELECT user_id, username FROM {table_name} WHERE dm_preference = TRUE")
dm_users = cur.fetchall()
for user_id, username in dm_users:
    print(f"  - {username} (ID: {user_id})")

cur.close()
conn.close()

print(f"\n✅ Setup complete! Now we have 2 birthdays today and multiple users with DM preferences enabled.")

🎂 Adding birthday for Kasane (ID: 417445162487119874) on 1995-09-03...
✅ Set birthday and enabled DM preference for Kasane
✅ Enabled DM preference for Piccar (ID: 305068083347718145)

📊 Current status:
  Husky (ID: 163035782259539968) - Birthday: 1998-09-03 (Age: 27), DM: True, Name: Klemens Drobnicki
  Kasane (ID: 417445162487119874) - Birthday: 1995-09-03 (Age: 30), DM: True, Name: Tim Kasane
  BirthdayBot-Dev (ID: 1358045590222541052) - Birthday: None, DM: False, Name: None None
  Piccar (ID: 305068083347718145) - Birthday: None, DM: True, Name: None None

🎉 Today's birthdays (September 3rd):
  🎂 Husky (ID: 163035782259539968) - Klemens Drobnicki turns 27, DM enabled: True
  🎂 Kasane (ID: 417445162487119874) - Tim Kasane turns 30, DM enabled: True

📧 Users with DM enabled:
  - Husky (ID: 163035782259539968)
  - Kasane (ID: 417445162487119874)
  - Piccar (ID: 305068083347718145)

✅ Setup complete! Now we have 2 birthdays today and multiple users with DM preferences enabled.


: 

In [ ]:
# Add another user with DM preferences enabled for testing
conn = get_db_connection()
cur = conn.cursor()

print("🔧 Adding DM preference for another user to test notifications...")

# Let's enable DM for Piccar as well
piccar_id = 305068083347718145  # Piccar's user ID from earlier output
cur.execute(f"UPDATE {table_name} SET dm_preference = TRUE WHERE user_id = %s", (piccar_id,))
conn.commit()
print(f"✅ Enabled DM preference for Piccar (ID: {piccar_id})")

# Verify the changes
cur.execute(f"SELECT user_id, username, dm_preference FROM {table_name} WHERE dm_preference = TRUE")
dm_enabled_users = cur.fetchall()
print(f"\n📧 Users with DM preferences enabled ({len(dm_enabled_users)}):")
for user_id, username, dm_pref in dm_enabled_users:
    print(f"  - {username} (ID: {user_id})")

# Now test the DM logic again
print(f"\n🧪 Testing DM logic with multiple users:")
birthday_user_id = 163035782259539968  # Husky's ID
print(f"Birthday user: {birthday_user_id}")

for user_id, username, dm_pref in dm_enabled_users:
    if user_id != birthday_user_id:
        print(f"  ✅ Would send DM to {username} (ID: {user_id})")
    else:
        print(f"  ❌ Skipping DM to birthday user {username} (ID: {user_id})")

cur.close()
conn.close()

In [ ]:
# Check specifically for September 3rd
cur = conn.cursor()
print("Checking for September 3rd birthdays:")
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname 
    FROM {table_name} 
    WHERE EXTRACT(MONTH FROM birthday) = 9 
    AND EXTRACT(DAY FROM birthday) = 3
""")
sep_3_birthdays = cur.fetchall()
print(f"Found {len(sep_3_birthdays)} birthdays on September 3rd:")
for row in sep_3_birthdays:
    print(f"  {row}")

# Check specifically for September 7th 
print("\nChecking for September 7th birthdays:")
cur.execute(f"""
    SELECT user_id, username, birthday, firstname, lastname 
    FROM {table_name} 
    WHERE EXTRACT(MONTH FROM birthday) = 9 
    AND EXTRACT(DAY FROM birthday) = 7
""")
sep_7_birthdays = cur.fetchall()
print(f"Found {len(sep_7_birthdays)} birthdays on September 7th:")
for row in sep_7_birthdays:
    print(f"  {row}")

cur.close()
conn.close()

print(f"\n🔍 DIAGNOSIS: The birthday in the database is September 7th (1998-09-07), not September 3rd (1998-09-03)!")
print("If you expected a birthday notification today (Sep 3), the birthday date needs to be corrected to 1998-09-03.")

## View current users and their DM preferences

In [10]:
try:
    with conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT user_id, username, birthday, dm_preference
                FROM birthdays
                ORDER BY username
            """)
            results = cur.fetchall()
            
            print("Current users and their DM preferences:")
            for row in results:
                print(f"User: {row[1]}, DM Preference: {row[3]}")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    # Get a fresh connection if needed
    conn = get_db_connection()

Current users and their DM preferences:
User: Abends, DM Preference: False
User: Albarn, DM Preference: False
User: AlexDieNase, DM Preference: False
User: Altdolphin, DM Preference: False
User: Bananu, DM Preference: False
User: Bubbels, DM Preference: False
User: Busten, DM Preference: False
User: D.Lightman, DM Preference: False
User: Dana, DM Preference: False
User: Der Fiebich, DM Preference: False
User: Error404, DM Preference: False
User: Error404-StreamingAudio, DM Preference: False
User: Falconslab, DM Preference: False
User: Flo/ Kneder, DM Preference: False
User: Frau Dr Bob, DM Preference: False
User: Grufnax, DM Preference: False
User: Hec, DM Preference: False
User: Holländer, DM Preference: False
User: Husky, DM Preference: False
User: Jay, DM Preference: False
User: Kasane, DM Preference: False
User: Keeper34, DM Preference: False
User: KnownAsMage, DM Preference: False
User: Lasir, DM Preference: False
User: Leviathuna, DM Preference: False
User: Markus, DM Preference:

## Initialize all DM preferences to False

In [11]:
try:
    with conn:
        with conn.cursor() as cur:
            cur.execute("""
                UPDATE birthdays
                SET dm_preference = FALSE
                WHERE dm_preference IS NULL
            """)
            rows_updated = cur.rowcount
            print(f"{rows_updated} users had their DM preferences initialized to False")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    conn = get_db_connection()

0 users had their DM preferences initialized to False


## Verify the changes

In [ ]:
try:
    with conn:
,
                SELECT COUNT(*) as total,
                       SUM(CASE WHEN dm_preference = TRUE THEN 1 ELSE 0 END) as dm_enabled,
                       SUM(CASE WHEN dm_preference = FALSE THEN 1 ELSE 0 END) as dm_disabled
                FROM birthdays
            """)
            stats = cur.fetchone()
            
            print(f"Total users: {stats[0]}")
            print(f"Users with DMs enabled: {stats[1]}")
            print(f"Users with DMs disabled: {stats[2]}")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    conn = get_db_connection()

In [12]:
# Close the connection when done
try:
    conn.close()
except:
    pass  # Connection might already be closed